### P0 WebAssembly Code Generator Tests


Total 11 test cases for WebAssembly code generator module
- 1 test case for statement expected
- 1 test case for no nested procedures
- 1 test case for no structured value parameter
- 1 test case for no structured result parameter
- 1 test case for assign
- 1 test case for input ans output
- 1 test case for parameter passing
- 1 test case for arrays and records
- 1 test case for booleans and conditions
- 1 test case for constant folding, local and global variables
- 1 test case for procedures

In [3]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runpywasm(wasmfile):
    import pywasm
    def writeInt(s, i): print(i)
    def writeFloat(s, i): print(i)
    def writeDouble(s, i): print(i)
    def writeln(s): print('\n')
    def readInt(s): return int(input())
    def readDouble(s): return float(input())
    def readFloat(s): return float(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'writeInt': writeInt, 'writeFloat': writeFloat, 'writeDouble': writeDouble,\
                                          'writeln': writeln, 'readInt': readInt, 'readFloat': readFloat, 'readDouble': readDouble}})

### Error: statement expected

In [4]:
compileString("""
program p
  var a: [1..10] → double
""", target = 'wat')

line 3 pos 25 statement expected


### Error: no nested procedures

In [5]:
compileString("""
program p
  procedure q()
    write(5.3)
  q(7)
""", target = 'wat')

line 4 pos 4 WASM: no nested procedures


### Error: no structured value parameter

In [6]:
compileString("""
type A = [1..10] → integer
procedure q(f: A)
  write(5)
program p
  writeln()
""")

(module
(import "P0lib" "writeInt" (func $writeInt (param i32)))
(import "P0lib" "writeDouble" (func $writeDouble (param f64)))
(import "P0lib" "writeFloat" (func $writeFloat (param f32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "readInt" (func $readInt (result i32)))
(import "P0lib" "readDouble" (func $readDouble (result f64)))
(import "P0lib" "readFloat" (func $readFloat (result f32)))
(func $q (param $f i32) 
(local $0 i32)
i32.const 5
call $writeInt
)
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $0 i32)
call $writeln
)
(memory 1)
(start $program)
)


### Error: no structured result parameter

In [7]:
compileString("""
type A = [1..10] → integer
procedure q() → (f: A)
  write(5)
program p
  writeln()
""")

(module
(import "P0lib" "writeInt" (func $writeInt (param i32)))
(import "P0lib" "writeDouble" (func $writeDouble (param f64)))
(import "P0lib" "writeFloat" (func $writeFloat (param f32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "readInt" (func $readInt (result i32)))
(import "P0lib" "readDouble" (func $readDouble (result f64)))
(import "P0lib" "readFloat" (func $readFloat (result f32)))
(func $q  (result i32)
(local $f i32)
(local $0 i32)
(local $_mp i32)
global.get $_memsize
local.set $_mp
global.get $_memsize
local.tee $f
i32.const 40
i32.add
global.set $_memsize
i32.const 5
call $writeInt
local.get $_mp
global.set $_memsize
local.get $f
)
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $0 i32)
call $writeln
)
(memory 1)
(start $program)
)


In [8]:
compileString("""
procedure q() → (x, y: float)
  x := y
program p
  writeln()
""", target = 'wat')

(module
(import "P0lib" "writeInt" (func $writeInt (param i32)))
(import "P0lib" "writeDouble" (func $writeDouble (param f64)))
(import "P0lib" "writeFloat" (func $writeFloat (param f32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "readInt" (func $readInt (result i32)))
(import "P0lib" "readDouble" (func $readDouble (result f64)))
(import "P0lib" "readFloat" (func $readFloat (result f32)))
(func $q  (result f32) (result f32)
(local $x f32)
(local $y f32)
(local $0 i32)
local.get $y
local.set $x
local.get $x
local.get $y
)
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $0 i32)
call $writeln
)
(memory 1)
(start $program)
)


### Assign

In [9]:
compileString("""
var a: [2 .. 9] → integer
program p
  var x, y: integer
    a[3] := 5
    x, y := a[3], 7
    x, y := y, x
    write(x); write(y) // writes 7, 5
""", 'assign.wat', target = 'wat')

In [10]:
!wat2wasm assign.wat

In [11]:
runpywasm('assign.wasm')

7
5


### Input and Output

In [14]:
compileString("""
program p
  var x: float
    x ← read(); x := 3 × x
    write(x); writeln()
    write(x × 5)
""", 'writefloat.wat', target = 'wat')

In [15]:
!wat2wasm writefloat.wat

In [17]:
compileString("""
program p
  var x: double
    x ← read(); x := 3 × x
    write(x); writeln()
    write(x × 5)
""", 'writedouble.wat', target = 'wat')

In [18]:
!wat2wasm writedouble.wat

### Parameter passing

In [19]:
compileString("""
type T = [1..10] → integer
var a: T
procedure q(b: integer, c: integer)
    write(b); write(c)
procedure r() → (d: integer)
    a[3] := 9; d := 5
program p
  var x: integer
  a[2] := 7; q(3, a[2]) // writes 3, 7
  x ← r(); write(x); write(a[3]) // writes 5, 9
""")

(module
(import "P0lib" "writeInt" (func $writeInt (param i32)))
(import "P0lib" "writeDouble" (func $writeDouble (param f64)))
(import "P0lib" "writeFloat" (func $writeFloat (param f32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "readInt" (func $readInt (result i32)))
(import "P0lib" "readDouble" (func $readDouble (result f64)))
(import "P0lib" "readFloat" (func $readFloat (result f32)))
(func $q (param $b i32) (param $c i32) 
(local $0 i32)
local.get $b
call $writeInt
local.get $c
call $writeInt
)
(func $r  (result i32)
(local $d i32)
(local $0 i32)
i32.const 8
i32.const 9
i32.store
i32.const 5
local.set $d
local.get $d
)
(global $_memsize (mut i32) i32.const 40)
(func $program
(local $x i32)
(local $0 i32)
i32.const 4
i32.const 7
i32.store
i32.const 3
i32.const 4
i32.load
call $q
call $r
local.set $x
local.get $x
call $writeInt
i32.const 8
i32.load
call $writeInt
)
(memory 1)
(start $program)
)


In [20]:
compileString("""
type T = [1..10] → integer
var a: T
procedure q(b: integer, c: integer)
    write(b); write(c)
procedure r() → (d: integer)
    a[3] := 9; d := 5
program p
  var x: integer
  a[2] := 7; q(3, a[2]) // writes 3, 7
  x ← r(); write(x); write(a[3]) // writes 5, 9
""", 'params.wat', target = 'wat')

In [21]:
!wat2wasm params.wat

In [22]:
runpywasm('params.wasm')

3
7
5
9


### Arrays and Records

In [23]:
compileString("""
type A = [1 .. 7] → integer
type R = (f: integer, g: A, h: integer)
var v: A
var w: R
var x: integer
program p
  x := 9;
  w.h := 12 - 7; write(w.h) // writes 5
  v[1] := 3; write(v[x - 8]) //writes 3
  w.g[x div 3] := 9; write(w.g[3]) // writes 9
  v[x - 2] := 7; w.g[x - 3] := 7
  write(v[7]); write(w.g[6]) // writes 7, 7
""", 'arrayrec.wat', target = 'wat')
!wat2wasm arrayrec.wat
runpywasm('arrayrec.wasm')

5
3
9
7
7


### Booleans and conditions

In [24]:
compileString("""
program p
  const five = 5
  const seven = 7
  const always = true
  const never = false
  var x, y, z: integer
  var b, t, f: boolean
    x := seven; y := 9; z := 11; t := true; f := false
    if true then write(7) else write(9)    // writes 7
    if false then write(7) else write(9)   // writes 9
    if t then write(7) else write(9)       // writes 7
    if f then write(7) else write(9)       // writes 9
    if ¬ t then write(7) else write(9)     // writes 9
    if ¬ f then write(7) else write(9)     // writes 7
    if t or t then write(7) else write(9)  // writes 7
    if t or f then write(7) else write(9)  // writes 7
    if f or t then write(7) else write(9)  // writes 7
    if f or f then write(7) else write(9)  // writes 9
    if t and t then write(7) else write(9) // writes 7
    if t and f then write(7) else write(9) // writes 9
    if f and t then write(7) else write(9) // writes 9
    if f and f then write(7) else write(9) // writes 9
    writeln()
    b := true
    if b then write(3) else write(5) // writes 3
    b := false
    if b then write(3) else write(5) // writes 5
    b := x < y
    if b then write(x) else write(y) // writes 7
    b := (x > y) or t
    if b then write(3) else write(5) // writes 3
    b := (x > y) or f
    if b then write(3) else write(5) // writes 5
    b := (x = y) or (x > y)
    if b then write(3) else write(5) // writes 5
    b := (x = y) or (x < y)
    if b then write(3) else write(5) // writes 3
    b := f and (x ≥ y)
    if b then write(3) else write(5) // writes 5
    writeln()
    while y > 3 do                   // writes 9, 8, 7, 6, 5, 4
      write(y); y := y - 1
    write(y); writeln()              // writes 3
    if ¬(x < y) and t then
      write(x)                       // writes 7
""", 'cond.wat', target = 'wat')

In [25]:
!wat2wasm cond.wat

In [26]:
runpywasm('cond.wasm')

7
9
7
9
9
7
7
7
7
9
7
9
9
9


3
5
7
3
5
5
3
5


9
8
7
6
5
4
3


7


### Constant Folding, Local & Global Variables

In [27]:
compileString("""
const seven = (9 mod 3 + 5 × 3) div 2
type int = integer
var x, y: integer
procedure q()
  const sotrue = true and true
  const sofalse = false and true
  const alsotrue = false or true
  const alsofalse = false or false
  var x: int
    x := 3
    if sotrue then y := x else y := seven
    write(y) // writes 3
    if sofalse then y := x else y := seven
    write(y) // writes 7
    if alsotrue then y := x else y := seven
    write(y) // writes 3
    if alsofalse then y := x else y := seven
    write(y) // writes 7
    if ¬(true or false) then write(5) else write(9)
program p
  x := 7; q(); write(x) // writes 7
""", 'folding.wat', target = 'wat')

In [28]:
!wat2wasm folding.wat

In [29]:
runpywasm('folding.wasm')

3
7
3
7
9
7


### Procedures

In [30]:
compileString("""
var g: integer          // global variable
procedure q(v: integer) // value parameter
  var l: integer        // local variable
    l := 9
    if l > v then write(l)
    else write(g)
program p
  g := 5; q(7)
""", 'proc.wat', target = 'wat')

In [31]:
!wat2wasm proc.wat

In [32]:
runpywasm('proc.wasm')

9
